# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [4]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path, index_col=[0])

# Display sample data
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Alfred,ZA,-33.5906,26.8910,60.19,95,100,8.01,moderate rain
1,Pauini,BR,-7.7136,-66.9764,87.30,41,95,3.65,overcast clouds
2,Port Alfred,ZA,-33.5906,26.8910,60.19,95,100,10.00,moderate rain
3,Pauini,BR,-7.7136,-66.9764,87.30,41,95,3.65,overcast clouds
4,Longyearbyen,SJ,78.2186,15.6401,39.04,87,20,0.00,few clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [5]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [6]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) &
                                      (city_data_df['Max Temp']<=max_temp)]
                                    
# Display sample data
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Alfred,ZA,-33.5906,26.8910,60.19,95,100,8.01,moderate rain
1,Pauini,BR,-7.7136,-66.9764,87.30,41,95,3.65,overcast clouds
2,Port Alfred,ZA,-33.5906,26.8910,60.19,95,100,10.00,moderate rain
3,Pauini,BR,-7.7136,-66.9764,87.30,41,95,3.65,overcast clouds
5,Samarai,PG,-10.6167,150.6667,82.90,73,22,8.68,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [7]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Alfred,ZA,-33.5906,26.8910,60.19,95,100,8.01,moderate rain
1,Pauini,BR,-7.7136,-66.9764,87.30,41,95,3.65,overcast clouds
2,Port Alfred,ZA,-33.5906,26.8910,60.19,95,100,10.00,moderate rain
3,Pauini,BR,-7.7136,-66.9764,87.30,41,95,3.65,overcast clouds
5,Samarai,PG,-10.6167,150.6667,82.90,73,22,8.68,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [8]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[['City','Country','Max Temp','Current Description','Lat','Lng']]

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
0,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910
1,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764
2,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910
3,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764
5,Samarai,PG,82.90,few clouds,-10.6167,150.6667


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [14]:
len(hotel_df.columns)

6

In [17]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df.insert(len(hotel_df.columns),"Hotel Name",np.nan)
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910,NaN
1,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764,NaN
2,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910,NaN
3,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764,NaN
5,Samarai,PG,82.90,few clouds,-10.6167,150.6667,NaN


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [18]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [19]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910,The Halyards Hotel
1,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764,Casa do Kiwliks
2,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910,The Halyards Hotel
3,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764,Casa do Kiwliks
5,Samarai,PG,82.90,few clouds,-10.6167,150.6667,Nuli Sapi


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [21]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910,The Halyards Hotel
1,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764,Casa do Kiwliks
2,Port Alfred,ZA,60.19,moderate rain,-33.5906,26.8910,The Halyards Hotel
3,Pauini,BR,87.30,overcast clouds,-7.7136,-66.9764,Casa do Kiwliks
5,Samarai,PG,82.90,few clouds,-10.6167,150.6667,Nuli Sapi


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [22]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [23]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [24]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>The Halyards Hotel</dd>\n<dt>City</dt><dd>Port Alfred</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>moderate rain and 60.19 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Casa do Kiwliks</dd>\n<dt>City</dt><dd>Pauini</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 87.3 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The Halyards Hotel</dd>\n<dt>City</dt><dd>Port Alfred</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>moderate rain and 60.19 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Casa do Kiwliks</dd>\n<dt>City</dt><dd>Pauini</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 87.3 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Nuli Sapi</dd>\n<dt>City</dt><dd>Samarai</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Weather</dt><dd>few clouds and 82.9 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sina Hotel</dd>\n<dt>City</dt><dd

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [25]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-33.5906,26.8910
1,-7.7136,-66.9764
2,-33.5906,26.8910
3,-7.7136,-66.9764
5,-10.6167,150.6667
6,34.3142,47.0650
7,37.1500,122.3833
8,-5.1989,-35.4608
15,-13.7000,-76.2167
16,-0.7393,-90.3518


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [26]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# YOUR CODE HERE

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))